In [1]:
import pandas as pd 
import numpy as np
import json, urllib.request
from datetime import datetime
from datetime import timedelta
from collections import Counter
import calendar

transaction_data = pd.read_json('transaction-data-adhoc-analysis.json')
months_loop = [pd.to_datetime(x).strftime("%m") for x in list(transaction_data['transaction_date'])]
#repeaters = f.open("repeaters.json","w")
#transaction_data.drop('Months', axis=1, inplace=True)
transaction_data.insert(2,"Months",months_loop, True)

In [5]:
#repeaters shorter
month_list_rep = transaction_data['Months']
min_month_rep = int(month_list_rep.min())
max_month_rep = int(month_list_rep.max())
prev_month_rep = min_month_rep - 1
current_month_rep = prev_month_rep + 1
month_repeaters_count = []
while prev_month_rep < max_month_rep:
    if prev_month_rep < 10:
        prev_month_rep_str = "0" + str(prev_month_rep)
    else:
        prev_month_rep_str = str(prev_month_rep)
        
    if current_month_rep < 10:
        current_month_rep_str = "0"+str(current_month_rep)
    else:
        current_month_rep_str = str(current_month_rep)
        
    month_one_names = transaction_data.loc[transaction_data['Months']== prev_month_rep_str,'name']
    month_two_names = transaction_data.loc[transaction_data['Months']== current_month_rep_str,'name']
    month_repeaters = len(pd.Series(list(set(month_one_names) & set(month_two_names)), dtype=pd.StringDtype()).unique())
    prev_month_rep = prev_month_rep + 1
    current_month_rep = current_month_rep + 1
    month_repeaters_count.append(month_repeaters)
    
month_repeaters_count

[0, 5172, 5216, 5154, 5110, 5193]

In [7]:
#inactive function
month_list_in = transaction_data['Months']
min_month_in = int(month_list_in.min())
max_month_in = int(month_list_in.max())
names_set_in = transaction_data["name"].unique()
prev_month_in = min_month_in - 1
current_month_in = prev_month_in + 1
month_inactive_count = []
while prev_month_in < max_month_in:
    if prev_month_in < 10:
        prev_month_in_str = "0"+str(prev_month_in)
    else:
        prev_month_in_str = str(prev_month_in)
        
    if current_month_in < 10:
        current_month_in_str = "0"+str(current_month_in)
    else:
        current_month_in_str = str(current_month_in)
        
    current_month_names = transaction_data.loc[transaction_data['Months']== current_month_in_str,'name']
    
    if min_month_in == current_month_in or max_month_in <= prev_month_in:
        inactive_month = 0
    else:
        inactive_month = len(pd.Series(list(set(names_set_in).difference(current_month_names))))
    
    month_inactive_count.append(inactive_month)
    prev_month_in = prev_month_in + 1
    current_month_in = current_month_in + 1

month_inactive_count

[0, 1856, 1865, 1931, 1919, 1835]

In [8]:
#engaged shortcut code
month_list_eng = transaction_data['Months']
engaged_users_array = []
min_month_eng = int(month_list_eng.min())
max_month_eng = int(month_list_eng.max())
prev_engaged_month = min_month_eng - 1
current_month_eng = min_month_eng

while prev_engaged_month < max_month_eng:
    if prev_engaged_month < 10:
        prev_engaged_month_str = "0"+str(prev_engaged_month)
    else:
        prev_engaged_month_str = str(prev_engaged_month)
        
    if current_month_eng < 10:
        current_month_eng_str = "0"+str(current_month_eng)
    else:
        current_month_eng_str = str(current_month_eng)

    if prev_engaged_month < min_month_eng:
        engaged_users_append = 0
    elif prev_engaged_month == min_month_eng:
        prev_eng = transaction_data.loc[transaction_data['Months'] == prev_engaged_month_str,'name']
        current_eng = transaction_data.loc[transaction_data['Months'] == current_month_eng_str,'name']
        current_month_engaged = pd.Series(list(set(prev_eng) & set(current_eng)), dtype=pd.StringDtype()).unique()
        engaged_users = current_month_engaged
        engaged_users_append = len(engaged_users)
    else:
        prev_eng = transaction_data.loc[transaction_data['Months'] == prev_engaged_month_str,'name']
        current_eng = transaction_data.loc[transaction_data['Months'] == current_month_eng_str,'name']
        current_month_engaged = pd.Series(list(set(prev_eng) & set(current_eng)), dtype=pd.StringDtype()).unique()
        engaged_users = pd.Series(list(set(current_month_engaged) & set(engaged_users)), dtype=pd.StringDtype()).unique()
        engaged_users_append = len(engaged_users)
        
    engaged_users_array.append(engaged_users_append)
    prev_engaged_month = prev_engaged_month + 1
    current_month_eng = current_month_eng + 1
    

engaged_users_array


[0, 5172, 4126, 3289, 2667, 2190]

In [3]:
months_for_table = []
month_list_tab = transaction_data['Months']
min_month_tab = int(month_list_tab.min())
max_month_tab = int(month_list_tab.max())
current_month_tab = min_month_tab

while current_month_tab <= max_month_tab:
    month_name = calendar.month_name[current_month_tab]
    months_for_table.append(month_name)
    current_month_tab = current_month_tab + 1

months_for_table

['January', 'February', 'March', 'April', 'May', 'June']

In [66]:
table_array_2 = [months_for_table, month_repeaters_count, month_inactive_count, engaged_users_array]
counts = [month_repeaters_count, month_inactive_count, engaged_users_array]
df_tab_2 = pd.DataFrame(table_array_2, index=['','Repeaters','Inactive','Engaged'])
df_tab_2

,0,1,2,3,4,5
,January,February,March,April,May,June
Repeaters,0,5172,5216,5154,5110,5193
Inactive,0,1856,1865,1931,1919,1835
Engaged,0,5172,4126,3289,2667,2190
